# Cape Basin Super Resolution

A simple notebook to test assorted SR models operating on the Cape Basin dataset

## Preparation

In [1]:
%matplotlib ipympl
import torch
import numpy as np
from fmod.base.gpu import set_device, get_device
import hydra, os
from fmod.base.util.config import fmconfig, cfg, ConfigContext
from fmod.model.sres.manager import ResultsAccumulator
from fmod.base.io.loader import TSet
from fmod.base.util.ops import fmbdir, fmtp
from typing import Any, Dict, List, Tuple, Type, Optional, Union
import matplotlib.pyplot as plt

In [2]:
task="sres"
models = [ 'dbpn' ]
dataset="LLC4320-v1"
scenario="s4"
fsize, nmodels = 4.0, len(models)
yscale = "log"
ccustom = {}

### Results data
Read results summary from previous runs:

### Plot Losses

In [ ]:
with plt.ioff():
	fig, axs = plt.subplots(nrows=nmodels, ncols=1, figsize=[fsize*2,fsize*nmodels], layout="tight")

if nmodels == 1:  axs = [axs]
fig.suptitle('SRes Loss Over Training Epochs', fontsize=14, va="top", y=1.0)
fmt = { TSet.Train: 'b',  TSet.Validation: 'g' }
for ax, model in zip(axs,models):
    with ConfigContext(task, model, dataset, scenario, ccustom) as cc:
        accum_results = ResultsAccumulator(task,dataset,scenario,model)
        accum_results.load_results()
        (x,y), min_loss =  accum_results.get_plot_data(), {}
        for tset in [ TSet.Train, TSet.Validation ]: 
            xp, yp = x[ tset ], y[ tset ]
            min_loss[tset] = yp.min() if (yp.size > 0) else 0.0
            ax.plot( xp, yp, fmt[tset], label=tset.name )
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Loss")
        ax.set_yscale(yscale)
        ax.set_title(f"Model '{model}':  Validation Loss = {min_loss[TSet.Validation]:.4f} ")
        ax.legend()
    
plt.show()